In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# import tensorflow as tf
# from keras.layers import LayerNormalization
print(tf.__version__)

## Clone Git Vocal

In [ ]:
!git clone https://github.com/xxichlas/explore-level3 # The github repo from our ML members, this is the same repo as the one in https://github.com/SIBISA-github/sibisa-ml

## **Cloning TFOD 2.0 Github**

In [ ]:
cd /content/explore-level3

In [ ]:
cd Tensorflow

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
pwd

In [ ]:
cd models/research

In [ ]:
pwd

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git


In [ ]:
cd cocoapi/PythonAPI

In [ ]:
!make

In [ ]:
cp -r pycocotools /content/explore-level3/Tensorflow/models/research

### Install the Object Detection API

In [ ]:
cd ..

In [ ]:
cd .. 

In [ ]:
pwd

In [ ]:
cp object_detection/packages/tf2/setup.py .

In [ ]:
## IMPORTANT 
## Because of the TF 2.9.0 update, TF OD currently doesn't work well with that version
## To solver this, before running the next cell
## go to the setup.py in /content/explore-vocal-phone/Tensorflow/models/research/setup.py from the files icon in the left pane
## and then change the files with this pastebin code https://pastebin.com/KCg6uK0h
## After that run the next cell TWO TIMES 


In [ ]:
!python -m pip install . #RUN 2 KALI BIAR KE INSTALL SEMUA

In [ ]:
# Test the API, From within TensorFlow/models/research/, you can skip this 
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
# Downgrade opencv, neccesary
!pip uninstall opencv-python-headless==4.5.5.64
! pip install opencv-python-headless==4.5.2.52

## Setup Path 

In [ ]:
pwd

In [ ]:
cd ..

In [ ]:
cd ..

In [ ]:
cd /content/explore-level3/Tensorflow/workspace

In [ ]:
!mkdir models
!mkdir annotations

In [ ]:
cd /content/explore-level3

In [ ]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

## Create Label Map

In [ ]:
## Check the /content/explore-vocal-phone/Tensorflow/workspace/annotations
## If there's an existing label_map.pbtxt, just deleted the file first and then run this cell
labels = [
    {'name':'A', 'id':1}, 
    {'name':'I', 'id':2},
    {'name':'aku', 'id':3},
    {'name':'bicara', 'id':4},
    {'name':'kamu', 'id':5}
    ]

with open(ANNOTATION_PATH + '/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

## Create TF Records

In [ ]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

## Copy Model Config to Training Folder


In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [ ]:
# Check if the /content/explore-vocal-phone/Tensorflow/workspace/models has files in it
# if exists, run this cell, if empty just run the next one
!rm -r /content/explore-level3/Tensorflow/workspace/models/my_ssd_mobnet/*

In [ ]:
!mkdir {'Tensorflow/workspace/models/'+CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

## Update Config

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [ ]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [ ]:
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = 5
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

## Train Model

In [ ]:
!python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=20000


In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime, os
tensorboard --logdir=/content/explore-kata-phone/Tensorflow/workspace/models/my_ssd_mobnet

In [ ]:
!zip -r /content/explore-level3/Tensorflow/workspace/models.zip /content/explore-level3/Tensorflow/workspace/models

In [ ]:
pwd

In [ ]:
!python /content/explore-level3/Tensorflow/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir /content/explore-level3/Tensorflow/workspace/models/my_ssd_mobnet \
    --output_directory /content/explore-level3/Tensorflow/workspace/saved \
    --pipeline_config_path /content/explore-level3/Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config

In [ ]:
!zip -r /content/explore-level3/Tensorflow/workspace/saved.zip /content/explore-level3/Tensorflow/workspace/saved

In [ ]:
!python /content/explore-level3/Tensorflow/models/research/object_detection/export_tflite_graph_tf2.py \
    --trained_checkpoint_dir /content/explore-level3/Tensorflow/workspace/models/my_ssd_mobnet \
    --output_directory /content/explore-level3/Tensorflow/workspace/Graph \
    --pipeline_config_path /content/explore-level3/Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config

In [ ]:
!zip -r /content/explore-level3/Tensorflow/workspace/graph.zip /content/explore-level3/Tensorflow/workspace/Graph

### Convert to TFlite

In [ ]:
!pip install tflite-support-nightly


In [ ]:
!sudo apt-get install libportaudio2


In [ ]:
pip install --upgrade flatbuffers==1.12


In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.utils import config_util
from object_detection.builders import model_builder

from tflite_support import flatbuffers
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb

%matplotlib inline

In [ ]:
_TFLITE_MODEL_PATH = "/content/explore-level3/Tensorflow/workspace/model.tflite"
saved_model_dir = '/content/explore-level3/Tensorflow/workspace/Graph/saved_model'

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open(_TFLITE_MODEL_PATH, 'wb') as f:
  f.write(tflite_model)

In [ ]:
_ODT_LABEL_MAP_PATH = '/content/explore-level3/Tensorflow/workspace/annotations/label_map.pbtxt'
_TFLITE_LABEL_PATH = "/content/explore-level3/Tensorflow/workspace/tflite_label_map.txt"

category_index = label_map_util.create_category_index_from_labelmap(
    _ODT_LABEL_MAP_PATH)
f = open(_TFLITE_LABEL_PATH, 'w')
for class_id in range(1, 6):
  if class_id not in category_index:
    f.write('???\n')
    continue
  name = category_index[class_id]['name']
  f.write(name+'\n')
f.close()

In [ ]:
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils

_TFLITE_MODEL_WITH_METADATA_PATH = "/content/explore-level3/Tensorflow/workspace/model_with_metadata.tflite"

writer = object_detector.MetadataWriter.create_for_inference(
    writer_utils.load_file(_TFLITE_MODEL_PATH), input_norm_mean=[127.5], 
    input_norm_std=[127.5], label_file_paths=[_TFLITE_LABEL_PATH])
writer_utils.save_file(writer.populate(), _TFLITE_MODEL_WITH_METADATA_PATH)

In [ ]:
from tflite_support import metadata

displayer = metadata.MetadataDisplayer.with_model_file(_TFLITE_MODEL_WITH_METADATA_PATH)
print("Metadata populated:")
print(displayer.get_metadata_json())
print("=============================")
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())

### Download all outputs

In [ ]:
from google.colab import files
files.download('/content/explore-level3/Tensorflow/workspace/models.zip') 

In [ ]:
from google.colab import files
files.download('/content/explore-level3/Tensorflow/workspace/saved.zip') 

In [ ]:
from google.colab import files
files.download('/content/explore-level3/Tensorflow/workspace/graph.zip') 

In [ ]:
from google.colab import files
files.download('/content/explore-level3/Tensorflow/workspace/model.tflite') 

In [ ]:
from google.colab import files
files.download('/content/explore-level3/Tensorflow/workspace/model_with_metadata.tflite') 